In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
# finetune and mount google drive, because i don't have linux machine
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from unsloth import FastLanguageModel
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import json
import torch
from torch.utils.data import Dataset
import random
from typing import List, Dict
from transformers import AutoTokenizer
from tqdm import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# download base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= "IlyaGusev/saiga_llama3_8b",
    dtype=torch.bfloat16,
    load_in_8bit=False,
    load_in_4bit=True,
    max_seq_length = max_seq_length,
    # attn_implementation="flash_attention_2",
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

Device does not support bfloat16. Will change to float16.


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#add LoRA adapters so we only need to update 1 to 10% of all parameters

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    modules_to_save = ["lm_head"],
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Casting lm_head to float32


In [ ]:
def read_jsonl(file_name):
    with open(file_name, encoding="utf-8") as r:
        return [json.loads(line) for line in r]
    
    
# download local dataset of questions and answers 
dataset_raw = read_jsonl("/content/raw_data.json")
dataset_raw = Dataset.from_list(dataset_raw)
# split to train and valid
train_dataset, validation_dataset= dataset_raw.train_test_split(test_size=0.1).values()

In [ ]:
# ChatDataset based on the IlyaGusev "Saiga"
class ChatDataset(Dataset):
    def __init__(
        self,
        original_records: List[Dict],
        tokenizer: AutoTokenizer,
        max_tokens_count: int,
        only_target_loss: bool = True,
        add_global_bos: bool = True,
        add_global_eos: bool = True,
        labels_pad_token_id: int = -100,
    ):
        self.original_records = original_records
        self.tokenizer = tokenizer
        self.max_tokens_count = max_tokens_count
        self.only_target_loss = only_target_loss
        self.labels_pad_token_id = labels_pad_token_id
        self.add_global_bos = add_global_bos
        self.add_global_eos = add_global_eos
        self.is_printed = False

        self.records = []
        for record in tqdm(original_records):
            tensors = self.convert_record(record)
            self.records.append(tensors)

    def __len__(self):
        return len(self.records)

    def __getitem__(self, index):
        return self.records[index]

    def get_tokens(self, messages):
        tokens = self.tokenizer.apply_chat_template(
            messages,
            add_special_tokens=False,
            tokenize=True,
            add_generation_prompt=False,
        )
        if tokens[0] == self.tokenizer.bos_token_id:
            tokens = tokens[1:]
        return tokens

    def convert_record(self, record):
        input_ids, labels = [], []

        for message in record["messages"]:
            message_input_ids = self.get_tokens([message])
            message_labels = message_input_ids
            if len(input_ids) + len(message_input_ids) > self.max_tokens_count - 2:
                break

            labels_mask = [
                self.labels_pad_token_id for _ in range(len(message_input_ids))
            ]
            if (
                message["role"] not in ("assistant", "bot", "gpt")
                and self.only_target_loss
            ):
                message_labels = labels_mask

            input_ids.extend(message_input_ids)
            labels.extend(message_labels)

        if not input_ids:
            return None

        if self.add_global_bos and input_ids[0] != self.tokenizer.bos_token_id:
            input_ids.insert(0, self.tokenizer.bos_token_id)
            labels.insert(0, self.labels_pad_token_id)

        if input_ids[-2] == self.tokenizer.eos_token_id:
            input_ids = input_ids[:-1]
            labels = labels[:-1]

        if self.add_global_eos and input_ids[-1] != self.tokenizer.eos_token_id:
            input_ids.append(self.tokenizer.eos_token_id)
            labels.append(self.tokenizer.eos_token_id)


        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(labels)
        attention_mask = input_ids.new_ones(input_ids.size())

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

In [ ]:
# create ChatDataset based on the IlyaGusev "Saiga"
datasets = []
for records in (train_dataset, validation_dataset):
    datasets.append(
        ChatDataset(
            records,
            tokenizer,
            max_tokens_count=2000,
            only_target_loss=True,  # If True, only compute loss for the target sequence
            add_global_bos=True,  # Add a global beginning-of-sequence token
            add_global_eos=True,  # Add a global end-of-sequence token
        )
    )

train_dataset, val_dataset = datasets
data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)

100%|██████████| 5/5 [00:00<00:00, 628.15it/s]


In [ ]:
# Configuration for the Trainer

trainer_config = {
    "evaluation_strategy": "steps",     # Evaluate the model every 'eval_steps' steps
    "per_device_train_batch_size": 1,   # Batch size for training on each device
    "per_device_eval_batch_size": 1,    # Batch size for evaluation on each device
    "gradient_accumulation_steps": 128, # Number of updates steps to accumulate before performing a backward/update pass
    "eval_steps": 4,                    # Number of update steps between two evaluations
    "save_steps": 4,                    # Number of updates steps before saving the model checkpoint
    "logging_steps": 1,                 # Number of update steps between two logs
    "learning_rate": 5e-5,              # Initial learning rate for the optimizer
    "num_train_epochs": 1,              # Total number of training epochs to perform
    "max_steps": 20,                    # Total number of training steps to perform. If set to -1, the number of steps is determined by the number of epochs
    "lr_scheduler_type": "cosine",      # The scheduler type to use for learning rate scheduling
    "warmup_steps": 4,                  # Number of steps used for a linear warmup from 0 to 'learning_rate'
    "bf16": torch.cuda.is_bf16_supported(),     # Use bf16 (if supported) for mixed precision training
    "fp16": not torch.cuda.is_bf16_supported(), # Use fp16 if bf16 is not supported for mixed precision training
    "optim": "adamw_8bit",                      # Optimizer to use (in this case, AdamW with 8-bit optimizations)
    "load_best_model_at_end": True,             # Whether to load the best model found during training at the end
    "save_total_limit": 1,                      # Limit the total amount of checkpoints. Deletes the older checkpoints
    "seed": 1488,                               # Random seed for reproducibility
    "max_grad_norm": 1.0,                       # Maximum gradient norm (for gradient clipping)
    "weight_decay": 0.05                        # Weight decay to apply (if any)
} 
output_dir = '/content'
trainer = Trainer(
                model=model,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                data_collator=data_collator,
                args=TrainingArguments(**trainer_config, output_dir=output_dir)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 45 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 128
\        /    Total batch size = 128 | Total steps = 20
 "-____-"     Number of trainable parameters = 567,279,616


Step,Training Loss,Validation Loss
4,0.145900,2.074281
8,0.153300,1.915339
12,0.166200,1.896950
16,0.168800,1.887999
20,0.223300,1.882977


In [ ]:
# Local saving adapter weights and tokenizer
model.save_pretrained("/content/drive/MyDrive/lora_adapter") 
tokenizer.save_pretrained("/content/drive/MyDrive/lora_adapter")

('/content/drive/MyDrive/lora_adapter/tokenizer_config.json',
 '/content/drive/MyDrive/lora_adapter/special_tokens_map.json',
 '/content/drive/MyDrive/lora_adapter/tokenizer.json')

In [ ]:
import gc
# Clear the GPU cache
torch.cuda.empty_cache()
# Run the garbage collection
gc.collect()

317

In [ ]:
# Save to q4_k_m GGUF
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.45 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:15<00:00,  4.22s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be ./model-unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rope theta = 500000.0
INFO:hf-to-gguf:gguf: rms norm epsilon =

In [ ]:
!cp '/content/model-unsloth.Q4_K_M.gguf' '/content/drive/MyDrive/llama_8b_gost.Q4_K_M.gguf'